In [2]:
"""
(C) Copyright IBM Corporation 2018

All rights reserved. This program and the accompanying materials
are made available under the terms of the Eclipse Public License v1.0
which accompanies this distribution, and is available at
http://www.eclipse.org/legal/epl-v10.html
"""

from collections import defaultdict

import numpy as np

from optparse import OptionParser

import balancing_gan as bagan
from rw.batch_generator import BatchGenerator as BatchGenerator
from utils import save_image_array

import os

In [3]:
# Read command line parameters
seed = 0
np.random.seed(seed)
unbalance = 0.05
gratio_mode = "uniform"
dratio_mode = "uniform"
gan_epochs = 150
adam_lr = 0.00005
opt_class = 0
batch_size = 128
dataset_name = 'MNIST'

# Set channels for mnist.
channels = 1 if dataset_name == 'MNIST' else 3
print('Using dataset: ', dataset_name)

# Result directory
res_dir = "./res_{}_dmode_{}_gmode_{}_unbalance_{}_epochs_{}_lr_{:f}_seed_{}".format(
    dataset_name, dratio_mode, gratio_mode, unbalance, gan_epochs, adam_lr, seed
)
if not os.path.exists(res_dir):
    os.makedirs(res_dir)

# Read initial data.
print("read input data...")
bg_train_full = BatchGenerator(BatchGenerator.TRAIN, batch_size,
                               class_to_prune=None, unbalance=None, dataset=dataset_name)
bg_test = BatchGenerator(BatchGenerator.TEST, batch_size,
                         class_to_prune=None, unbalance=None, dataset=dataset_name)

print("input data loaded...")

shape = bg_train_full.get_image_shape()

min_latent_res = shape[-2]
while min_latent_res > 8:
    min_latent_res = min_latent_res / 2
min_latent_res = int(min_latent_res)

classes = bg_train_full.get_label_table()

Using dataset:  MNIST
read input data...
input data loaded...


In [4]:
shape

[28, 28, 1]

In [8]:
# Initialize statistics information
gan_train_losses = defaultdict(list)
gan_test_losses = defaultdict(list)

img_samples = defaultdict(list)

# For all possible minority classes.
target_classes = np.array(range(len(classes)))
if opt_class >= 0:
    min_classes = np.array([opt_class])
else:
    min_classes = target_classes

for c in min_classes:
    # If unbalance is 1.0, then the same BAGAN model can be applied to every class because
    # we do not drop any instance at training time.
    if unbalance == 1.0 and c > 0 and (
        os.path.exists("{}/class_0_score.csv".format(res_dir, c)) and
        os.path.exists("{}/class_0_discriminator.h5".format(res_dir, c)) and
        os.path.exists("{}/class_0_generator.h5".format(res_dir, c)) and
        os.path.exists("{}/class_0_reconstructor.h5".format(res_dir, c))
    ):
        # Without additional imbalance, BAGAN does not need to be retrained, we simlink the pregenerated model
        os.symlink("{}/class_0_score.csv".format(res_dir), "{}/class_{}_score.csv".format(res_dir, c))
        os.symlink("{}/class_0_discriminator.h5".format(res_dir), "{}/class_{}_discriminator.h5".format(res_dir, c))
        os.symlink("{}/class_0_generator.h5".format(res_dir), "{}/class_{}_generator.h5".format(res_dir, c))
        os.symlink("{}/class_0_reconstructor.h5".format(res_dir), "{}/class_{}_reconstructor.h5".format(res_dir, c))

    # Unbalance the training set.
    bg_train_partial = BatchGenerator(BatchGenerator.TRAIN, batch_size,
                                      class_to_prune=c, unbalance=unbalance, dataset=dataset_name)

    # Train the model (or reload it if already available
    if not (
            os.path.exists("{}/class_{}_score.csv".format(res_dir, c)) and
            os.path.exists("{}/class_{}_discriminator.h5".format(res_dir, c)) and
            os.path.exists("{}/class_{}_generator.h5".format(res_dir, c)) and
            os.path.exists("{}/class_{}_reconstructor.h5".format(res_dir, c))
    ):
        # Training required
        print("Required GAN for class {}".format(c))

        print('Class counters: ', bg_train_partial.per_class_count)

        # Train GAN to balance the data
        gan = bagan.BalancingGAN(
            target_classes, c, dratio_mode=dratio_mode, gratio_mode=gratio_mode,
            adam_lr=adam_lr, res_dir=res_dir, image_shape=shape, min_latent_res=min_latent_res
        )
        gan.train(bg_train_partial, bg_test, epochs=gan_epochs)
        gan.save_history(
            res_dir, c
        )

    else:  # GAN pre-trained
        # Unbalance the training.
        print("Loading GAN for class {}".format(c))

        gan = bagan.BalancingGAN(target_classes, c, dratio_mode=dratio_mode, gratio_mode=gratio_mode,
                                 adam_lr=adam_lr, res_dir=res_dir, image_shape=shape, min_latent_res=min_latent_res)
        gan.load_models(
            "{}/class_{}_generator.h5".format(res_dir, c),
            "{}/class_{}_discriminator.h5".format(res_dir, c),
            "{}/class_{}_reconstructor.h5".format(res_dir, c),
            bg_train=bg_train_partial  # This is required to initialize the per-class mean and covariance matrix
        )

    # Sample and save images
    img_samples['class_{}'.format(c)] = gan.generate_samples(c=c, samples=10)

    save_image_array(np.array([img_samples['class_{}'.format(c)]]), '{}/plot_class_{}.png'.format(res_dir, c))

Required GAN for class 0
Class counters:  [338, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]


KeyboardInterrupt: 